In [ ]:
import os
import sys

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../../Python-Table/task2'))

import numpy as np
import matplotlib.pyplot as plt
import idx2numpy
import pandas as pd
from lib.Net import *
from lib.Func import *
from lib.net import *
from lib.fn import *

In [ ]:
# (batch_size, 1, 28, 28) -> (batch_size, 2, 24, 24) -> (batch_size, 2*24*24) -> (batch_size, 64) -> (batch_size, 10)
architecture = [
    {"module": Conv2d, "params": {"in_channels": 1, "out_channels": 2, "kernel_size": 5, "activation": "gelu"}},
    {"module": Flatten},
    {"module": Mlp, "params": {"input_dim": 1152, "output_dim": 64, "activation": "gelu"}},
    {"module": Dropout, "params": {"p": 0.3}},
    {"module": Mlp, "params": {"input_dim": 64, "output_dim": 10, "activation": "softmax"}},
]

In [ ]:
net = Net(architecture)

In [ ]:
datapath = "../data/MNIST/train-images.idx3-ubyte"
labelpath = "../data/MNIST/train-labels.idx1-ubyte"
modelpath = "../model/task2/MNIST.json"

In [ ]:
data = idx2numpy.convert_from_file(datapath)    # (60000, 28, 28)
data = np.expand_dims(data, axis=1)             # 添加通道维度 -> (60000, 1, 28, 28)

label = idx2numpy.convert_from_file(labelpath)
one_hot_labels = one_hot(label, 10)

In [ ]:
epochs = 1
pbar =tqdm(range(epochs))

for i in pbar:
    # data, one_hot_labels = shuffle(data, one_hot_labels)
    net.train(data, one_hot_labels, batch_size=512, lr=0.1, lossfunc="cross_entropy")

    y_hat = net.predict(data)
    loss = cross_entropy_loss(one_hot_labels, y_hat)
    y_hat = np.argmax(y_hat, axis=1)    # (60000, 10) -> (60000,)
    accuracy = np.mean(y_hat == one_hot_labels.argmax(axis=1))
    pbar.set_postfix({"loss": loss, "accuracy": f"{accuracy*100:.2f}%"})

In [ ]:
#net.save_params(modelpath_2)

In [ ]:
net_info = [
    # (1, 28, 28) -> (2, 24, 24)
    {"module": Conv2d, "param": {"in_channels": 1, "out_channels": 2, "kernel_size": 5}},
    {"module": Gelu},
    {"module": Flatten},
    {"module": Mlp, "param": {"in_features": 2 * 24 * 24, "out_features": 256}},
    {"module": Gelu},
    {"module": Dropout, "param": {"p": 0.3}},
    {"module": Mlp, "param": {"in_features": 256, "out_features": 10}},
    {"module": Softmax},
]

In [ ]:
net2 = Net(net_info)